In [ ]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [ ]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

In [ ]:
casos = casos_confirmados.get_casos()
casos.shape

In [ ]:
def get_est(mun):
    est = 'ERRO'
    municipios = static.municipios.loc[static.municipios['uf']!='PR']
    municipios['municipio_sesa'] = municipios['municipio_sesa'].apply(lambda x: normalize_hash(normalize_text(x)))
    municipios['municipio_ibge'] = municipios['municipio_ibge'].apply(lambda x: normalize_hash(normalize_text(x)))

    municipio = municipios.loc[municipios['municipio_sesa']==normalize_hash(mun)]
    if len(municipio) == 0:
        municipio = municipios.loc[municipios['municipio_ibge']==normalize_hash(mun)]

    if len(municipio) != 0:
        est = municipio.iloc[0]['uf']

    return est
    

In [ ]:
obitos = casos_confirmados.get_obitos()

obitos.loc[obitos['rs'].isnull(), 'uf'] = obitos.loc[obitos['rs'].isnull(), 'municipio'].apply(get_est)

obitos.loc[obitos['rs'].isnull(), 'municipio'] = obitos.loc[obitos['rs'].isnull(), 'municipio'] + '/' + obitos.loc[obitos['rs'].isnull(), 'uf']

obitos.to_excel(join(output,'obitos_mun_corrigidos.xlsx'))
obitos.loc[obitos['rs'].isnull()]

In [ ]:
obitos.loc[obitos.duplicated('hash')].append(obitos.loc[obitos['hash'].isin(obitos['hash_less'])],ignore_index=True).append(obitos.loc[obitos['hash'].isin(obitos['hash_more'])],ignore_index=True).to_excel(join(output,'obitos_duplicados.xlsx'))

In [ ]:
# obitos_nao_em_casos = obitos.loc[~obitos['hash'].isin(casos['hash'])]
# obitos_nao_em_casos.to_excel(join(output,'obitos_nao_em_casos.xlsx'))

# len(obitos_nao_em_casos)

# casos_idade_errada_less = obitos_nao_em_casos.loc[obitos['hash_less'].isin(casos['hash']), 'hash_less'].values.tolist()
# casos_idade_errada_more = obitos_nao_em_casos.loc[obitos['hash_more'].isin(casos['hash']), 'hash_more'].values.tolist()

# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'idade'] -=1

# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'hash'] = casos.loc[casos['hash'].isin(casos_idade_errada_less)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'hash_less'] = casos.loc[casos['hash'].isin(casos_idade_errada_less)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']-1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'hash_more'] = casos.loc[casos['hash'].isin(casos_idade_errada_less)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']+1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)


# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'idade'] +=1
# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'hash'] = casos.loc[casos['hash'].isin(casos_idade_errada_more)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'hash_less'] = casos.loc[casos['hash'].isin(casos_idade_errada_more)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']-1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'hash_more'] = casos.loc[casos['hash'].isin(casos_idade_errada_more)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']+1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)

# obitos.loc[~obitos['hash'].isin(casos['hash'])]

In [ ]:
exit()
duplicados = casos.loc[casos.duplicated('hash')].index.tolist()
duplicados_less = casos.loc[casos['hash'].isin(casos['hash_less'])].index.tolist()
duplicados_more = casos.loc[casos['hash'].isin(casos['hash_more'])].index.tolist()
dropar = set(duplicados + duplicados_less + duplicados_more)
casos.loc[dropar].to_excel('casos_duplicados.xlsx')
print(len(dropar))
casos = casos.drop(index=dropar)
len(casos)

In [ ]:
duplicados = casos.loc[casos.duplicated('hash')].index.tolist()
duplicados_less = casos.loc[casos['hash'].isin(casos['hash_less'])].index.tolist()
duplicados_more = casos.loc[casos['hash'].isin(casos['hash_more'])].index.tolist()
dropar = set(duplicados + duplicados_less + duplicados_more)
casos.loc[dropar].to_excel('obitos_duplicados_casos_confirmados.xlsx')
len(dropar)

In [ ]:
writer = pd.ExcelWriter("novos_casos_confirmados.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos.sort_values('nome').to_excel(writer,index=None)

writer.save()
writer.close()